In [24]:
#Set up the connection string:
#Need to include host, port, db, and username
creds = ''

In [106]:
#*************INTRO TO PostreSQL with Python *******************

#import psycopg2 module to communicate with PostgreSQL
import psycopg2
#The Error class handles any db error/exception that can occur
#Need this to make our application robust
from psycopg2 import Error

#stuff we learned in class
import pandas as pd

In [107]:
## ********* Laod PSQL DB ************ ###
try:
    #connect to database with credentials
    #creates a connection to a psql db instance
    #returns a psql connection object
    #And it is thread safe :)
    connection = psycopg2.connect(creds)
    #prints the postgreSQL Connection properties
    print(connection.get_dsn_parameters(),'\n')
    
    
    #creates a cursor object which allows us to execute psql
    #commands through Python
    #we can create as many cursors as we want from a single connection object
    #NOTE: any changes done to the db by a cursor are visible 
    #by other cursors, they are also not thread-safe
    #cursor encapsulates the query and can access rows at time
    cursor = connection.cursor()

    
    
    #We need a cursor to execute queries for us
    #Execute takes a SQL query string paramater.
    cursor.execute('SELECT version();')
    
    #OTher cursor methods: fetchone, fetchmany, fetchall
    record = cursor.fetchone()
    print('You are connected to - ', record, '\n')
except (Exception, psycopg2.Error) as error:
    print('Error while connecting to PostgreSQL ', error)
finally:
    #close connection to db
    if(connection):
        cursor.close()
        connection.close()
        print('PostgreSQL connection closed')

{'user': 'postgres', 'dbname': 'tcglocator', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.3, compiled by Visual C++ build 1914, 64-bit',) 

PostgreSQL connection closed


In [93]:
#############CREATE A TABLE################
def create_table():
    try:
        connection = psycopg2.connect(creds)
        cursor = connection.cursor()
        
        query = '''CREATE TABLE bay_area_smash_esports (
           EVENT     TEXT   NOT NULL,
           DATE      TEXT   NOT NULL,
           LOCATION  TEXT   NOT NULL,
           PRIMARY KEY (EVENT, DATE, LOCATION)); '''
        cursor.execute(query)
        #If everything is good to here, commit.
        connection.commit()
        print('Table created')
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while creating table ', error)
    finally:
        #close db connection
        if(connection):
            cursor.close()
            connection.close()
            print('Connection now closed')

In [94]:
create_table()

Table created
Connection now closed


In [95]:
#Execute a query to the given schema in the database
# returns every tuple in the table
def load_data(schema, table):
    try:
        connection = psycopg2.connect(creds)
        cursor = connection.cursor()
        sql_cmd = 'SELECT * FROM {}.{};'.format(str(schema), str(table))
        print('query: \n', sql_cmd)

        #Read query, returns a Pandas data frame
        data = pd.read_sql(sql_cmd, connection)
        return data
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error executing query: ', error)
    finally:
        #close db connection
        if(connection):
            cursor.close()
            connection.close()
            print('Connection now closed')

In [99]:
load_data('public','bay_area_smash_esports')

query: 
 SELECT * FROM public.bay_area_smash_esports;
Connection now closed


event             date        location
0       UCSC Smash  June 14th, 2019  Santa Cruz, CA
1       Surf Smash  July 15th, 2019  Santa Cruz, CA
2  Smash and Chill   May 15th, 2019    San Jose, CA

In [97]:
#**************** SQL Insert ****************
#input 3 strings
def create_tournament(eventname, date, location):
    try:
        connection = psycopg2.connect(creds)
        cursor = connection.cursor()
        query = "INSERT INTO public.bay_area_smash_esports VALUES ('" + eventname + "','" + date +"', '"+ location+ "');"
        cursor.execute(query)
        #After successful execution of query, commit changes to the db
        connection.commit()
        print('New row inserted')
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while inserting row ', error)
    finally:
        #close db connection
        if(connection):
            cursor.close()
            connection.close()
            print('Connection now closed')

In [98]:
create_tournament('UCSC Smash', 'June 14th, 2019', 'Santa Cruz, CA')
create_tournament('Surf Smash', 'July 15th, 2019', 'Santa Cruz, CA')
create_tournament('Smash and Chill', 'May 15th, 2019', 'San Jose, CA')

New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed


In [69]:
#**************** SQL Update ****************
def updateTable(location, eventname):
    try:
        connection = psycopg2.connect(creds)
        cursor = connection.cursor()

        print('Table before updating:')
        select_query = """SELECT * FROM public.bay_area_smash_esports where location = %s"""
        cursor.execute(select_query, (location,))
        record = cursor.fetchone()
        print(record)

        update_query = """UPDATE public.bay_area_smash_esports set event = %s where location =  %s"""
        cursor.execute(update_query, (eventname, location))
        connection.commit()
        count = cursor.rowcount
        print(count, ' records updated successfully')

        print('Table after updating record')
        select_query = """SELECT * FROM public.bay_area_smash_esports where location = %s"""
        cursor.execute(select_query, (location,))
        record = cursor.fetchone()
        print(record)
    except (Exception, psycopg2.Error) as error:
        print("Error in update operation", error)
    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("Connection closed")
    

In [100]:
location = 'Santa Cruz, CA'
eventName = 'Cruz Smash'
updateTable(location, eventName)

Table before updating:
('UCSC Smash', 'June 14th, 2019', 'Santa Cruz, CA')
2  records updated successfully
Table after updating record
('Cruz Smash', 'June 14th, 2019', 'Santa Cruz, CA')
Connection closed


In [72]:
#***************SQL Clear Table **************
def clear_table():
    try:
        connection = psycopg2.connect(creds)
        cursor = connection.cursor()
        query = "DELETE FROM public.bay_area_smash_esports;"
        cursor.execute(query)
        #After successful execution of query, commit changes to the db
        connection.commit()
        print('Table cleared')
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while trying to clear table')
    finally:
        #close db connection
        if(connection):
            cursor.close()
            connection.close()
            print('Connection now closed')

In [73]:
clear_table()

Table cleared
Connection now closed


In [142]:
#***************SQL UPDATE MULTIPLE ROWS WITH SINGLE QUERY **************
import psycopg2
##Update multiple rows of PostgreSQL table using a single Query in Python
#records is a list of tuples
#For each tupe the first value is the one to be replaced, with the second.
#executemany(querystring, list of tuples)
def updateInBulk(records):
    try:
        connection = psycopg2.connect(creds)
        cursor = connection.cursor()
        query="""UPDATE public.bay_area_smash_esports SET event = %s where location = %s"""
        cursor.executemany(query, records)
        connection.commit()
        row_count = cursor.rowcount
        print(row_count, "records updated")
    except (Exception, psycop2g.Error) as error:
        print("Error while updating table ", error)
    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [141]:
tuples = [('Melee 2v2', 'Santa Cruz, CA'), ('Silicon Valley Smash', 'San Jose, CA')]
updateInBulk(tuples)

0 records updated
PostgreSQL connection is closed


In [42]:
#***************SQL DELETE MULTIPLE ROWS WITH SINGLE QUERY **************
import psycopg2
def deleteInBulk(records):
    try:
        connection = psycopg2.connect(con_str)
        cursor = connection.cursor()
        query = """DELETE FROM public.bay_area_smash_esports where location = %s"""
        cursor.executemany(query, records)
        connection.commit()
        row_count = cursor.rowcount
        print(row_count, 'records deleted')
    except (Exception, psycopg2.Error) as error:
        print('Error while deleting in bulk ', error)
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("Connection closed")

In [43]:
tuples = [('Santa Cruz, CA', ), ('San Francisco, CA', )]
deleteInBulk(tuples)

4 records deleted
Connection closed


In [56]:
url = 'https://smash.gg/tournaments?per_page=30&filter=%7B%22upcoming%22%3Atrue%2C%22videogameIds%22%3A1386%2C%22distance%22%3A%2250mi%22%2C%22distanceFrom%22%3A%2237.7749295%2C-122.41941550000001%22%2C%22distanceFromCity%22%3A%22San%20Francisco%22%7D&page=1'

In [101]:
#Scraping a JS-Rendered Page with Selenium and PhantomJS
from selenium import webdriver
driver = webdriver.PhantomJS(executable_path = "C:/Users/Moises/Desktop/phantomjs")
response = driver.get(url)

In [102]:
tourney_names = []
tourney_locations_numattendies = []
tourney_dates = []
#tourney_link = []
tourney_types = []

#Returns a list of web-elements
p_element = driver.find_elements_by_xpath('//*[@class="gg-component-reset TournamentCard Card Card--wide"]')
for s in p_element:
    tourney_names.append(s.find_element_by_class_name('TournamentCardHeading__title').text)
    tourney_dates.append(s.find_element_by_class_name('TournamentCardHeading__information').text)
    tourney_types.append(s.find_element_by_class_name('EventList__title').text)
    try:
        tourney_locations_numattendies.append(s.find_element_by_class_name('sgg2ar68').text)
    except:
        tourney_locations_numattendies.append('N/A')

#print('# tourney_locations_numattendies: ',len(tourney_locations_numattendies))
#print('# names: ',len(tourney_names))
#print('# dates: ',len(tourney_dates))

In [103]:
#Using regular expressions, strip each string up until ', CA'
#Add the new string into tourney_locations
import re
tourney_locations = []
for s in tourney_locations_numattendies:
    if s != 'N/A':
        # . means anything
        # + means any number of times
        # , CA  is what they all should end with
        s = re.search(".+, CA", s)
        tourney_locations.append(s.group())
    else:
        tourney_locations.append(s)
print(tourney_locations)

['Fremont, CA', 'San Francisco, CA', 'Pittsburg, CA', 'N/A', 'San Jose, CA', 'Vacaville, CA', 'Redwood City, CA', 'Alamo, CA', 'Dublin, CA', 'Fremont, CA', 'Santa Clara, CA', 'Los Altos, CA', 'San Francisco, CA']


In [104]:
import pandas as pd
#put our collected data in a pandas dataframe
df = pd.DataFrame({'Event':tourney_names,
                       'Location': tourney_locations,
                       'Date':tourney_dates})
#interesting info for pandas df.
print(df.info())

#finally print the dataframe
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
Event       13 non-null object
Location    13 non-null object
Date        13 non-null object
dtypes: object(3)
memory usage: 196.0+ bytes
None


Event           Location  \
0                                                Taro        Fremont, CA   
1                Bonus Stage #32: Smash Bros Ultimate  San Francisco, CA   
2                              Center Stage Smash #13      Pittsburg, CA   
3   Weds Night Fights x Oakland 2019 Spring 2019 S...                N/A   
4                                      FanimeCon 2019       San Jose, CA   
5                        Ultimate at Outer Limits #23      Vacaville, CA   
6                                  Redwood Stadium II   Redwood City, CA   
7                                    Smash to Do Good          Alamo, CA   
8                     Smash the Galaxy #11 (Ultimate)         Dublin, CA   
9                 Fremont's Finest VI (at GG eSports)        Fremont, CA   
10                          SCU Ultimate Bronco Brawl    Santa Clara, CA   
11                                Smash the Library 6      Los Altos, CA   
12                                  Rishov Test 54712  San Francisco, CA   

                       Date  
0            May 21st, 2019  
1            May 21st, 2019  
2            May 21st, 2019  
3        May 22nd-23rd 2019  
4        May 24th-27th 2019  
5            May 24th, 2019  
6            May 25th, 2019  
7            June 1st, 2019  
8            June 1st, 2019  
9            June 2nd, 2019  
10           June 2nd, 2019  
11           June 9th, 2019  
12  10/15/2019 - 02/01/2020

In [105]:
for i in range(0, len(tourney_names)):
    create_tournament(tourney_names[i], tourney_dates[i], tourney_locations[i])

New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
Error while inserting row  syntax error at or near "s"
LINE 1: ...TO public.bay_area_smash_esports VALUES ('Fremont's Finest V...
                                                             ^

Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
New row inserted
Connection now closed
